<a href="https://colab.research.google.com/github/Thadchayini/week_5_assignment/blob/main/Assignment_Week_5_Named_Entity_Extraction_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, padding="max_length", is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader

# Create data loaders
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=16)
validation_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=16)

In [ ]:
from transformers import AutoModelForTokenClassification

label_list = dataset["train"].features["ner_tags"].feature.names
# Load the pre-trained BERT model for token classification
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments

# ... (previous code)

# Load the pre-trained BERT model for token classification
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(label_list))

# Define training arguments
training_args = TrainingArguments(
    output_dir="./content/drive/MyDrive/idp_bootcamp/Week_5/results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./content/drive/MyDrive/idp_bootcamp/Week_5/logs",
    logging_steps=10
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],

    eval_dataset=tokenized_datasets["validation"]
)

# Train the model
trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

wandb: ERROR API key must be 40 characters long, yours was 37


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 

 ··········


## Dataset preparation for traning

In [ ]:
from transformers import AutoTokenizer
import torch
from torch.utils.data import DataLoader, Dataset

class NERDataset(Dataset):
    def __init__(self, tokenized_dataset):
        self.tokenized_dataset = tokenized_dataset

    def __len__(self):
        return len(self.tokenized_dataset)

    def __getitem__(self, idx):
        item = self.tokenized_dataset[idx]
        return {
            'input_ids': torch.tensor(item['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(item['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(item['labels'], dtype=torch.long)
        }

def prepare_data():
    # Load the CoNLL-2003 dataset
    dataset = load_dataset("conll2003")

    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

    # Get label list from dataset
    label_list = dataset["train"].features["ner_tags"].feature.names

    def tokenize_and_align_labels(examples):
        tokenized_inputs = tokenizer(
            examples["tokens"],
            truncation=True,
            is_split_into_words=True,
            padding='max_length',
            max_length=128,
            return_tensors="pt"  # Return PyTorch tensors
        )

        labels = []
        for i, label in enumerate(examples["ner_tags"]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:
                if word_idx is None:
                    label_ids.append(-100)
                elif word_idx != previous_word_idx:
                    label_ids.append(label[word_idx])
                else:
                    label_ids.append(-100)
                previous_word_idx = word_idx
            labels.append(label_ids)

        tokenized_inputs["labels"] = labels
        return tokenized_inputs

    # Tokenize datasets
    tokenized_datasets = dataset.map(
        tokenize_and_align_labels,
        batched=True,
        remove_columns=dataset["train"].column_names
    )

    return tokenized_datasets, len(label_list)


## Implement the model

In [ ]:
from transformers import BertForTokenClassification
import torch.nn as nn

class NERModel(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.bert = BertForTokenClassification.from_pretrained(
            'bert-base-cased',
            num_labels=num_labels
        )

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        return outputs

## Implement the traning loop

In [ ]:
## your code here
def train_model(model, train_dataloader, eval_dataloader, optimizer, scheduler, device, num_epochs=3):
    model.train()
    # your code here


## Implement the evaluation function

In [ ]:
%pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=93ede95441f1a9e3c0604cf330676a16a2491ba20488ae4b6447991e67dbc69f
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
from seqeval.metrics import classification_report

def evaluate_model(model, test_dataloader, id2label, device):
    model.eval()
    # Your code here

    # Print classification report
    print(classification_report(true_labels, predicted_labels))

## Implement the main() function

In [ ]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

def main():
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load and prepare dataset
    print("Loading and preparing dataset...")
    tokenized_datasets, num_labels = prepare_data()

    # Convert to custom Dataset objects
    train_dataset = NERDataset(tokenized_datasets["train"])
    eval_dataset = NERDataset(tokenized_datasets["validation"])
    test_dataset = NERDataset(tokenized_datasets["test"])

    # Create data loaders
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=32,  # Increased batch size
        shuffle=True
    )
    eval_dataloader = DataLoader(
        eval_dataset,
        batch_size=32
    )
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=32
    )

    # Model initialization
    model = NERModel(num_labels=num_labels)
    model.to(device)

    # Define optimizer with better parameters
    optimizer = AdamW(
        model.parameters(),
        lr=5e-5,  # Increased learning rate
        weight_decay=0.01,
        eps=1e-8
    )

    # Learning rate scheduler
    num_epochs = 3
    num_training_steps = num_epochs * len(train_dataloader)
    num_warmup_steps = num_training_steps // 10
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps
    )

    # Training
    print("Starting training...")
    train_model(model, train_dataloader, eval_dataloader, optimizer, scheduler, device, num_epochs=num_epochs)

    # Evaluation
    print("\nEvaluating on validation set...")
    id2label = {
        0: "O",
        1: "B-PER",
        2: "I-PER",
        3: "B-ORG",
        4: "I-ORG",
        5: "B-LOC",
        6: "I-LOC",
        7: "B-MISC",
        8: "I-MISC"
    }

    evaluate_model(model, eval_dataloader, id2label, device)

    return model

In [ ]:
model = main()

Using device: cuda
Loading and preparing dataset...


Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting training...
Epoch 1/3, Batch 100/439, Loss: 0.1172
Epoch 1/3, Batch 200/439, Loss: 0.0986
Epoch 1/3, Batch 300/439, Loss: 0.1658
Epoch 1/3, Batch 400/439, Loss: 0.0224
Epoch 1/3, Average Loss: 0.2401
Validation Loss: 0.0387
Epoch 2/3, Batch 100/439, Loss: 0.0092
Epoch 2/3, Batch 200/439, Loss: 0.0232
Epoch 2/3, Batch 300/439, Loss: 0.0149
Epoch 2/3, Batch 400/439, Loss: 0.0312
Epoch 2/3, Average Loss: 0.0247
Validation Loss: 0.0330
Epoch 3/3, Batch 100/439, Loss: 0.0039
Epoch 3/3, Batch 200/439, Loss: 0.0023
Epoch 3/3, Batch 300/439, Loss: 0.0008
Epoch 3/3, Batch 400/439, Loss: 0.0441
Epoch 3/3, Average Loss: 0.0105
Validation Loss: 0.0332

Evaluating on validation set...
              precision    recall  f1-score   support

         LOC       0.97      0.96      0.97      1837
        MISC       0.89      0.91      0.90       922
         ORG       0.93      0.94      0.93      1341
         PER       0.97      0.98      0.97      1836

   micro avg       0.95      0.95     

## Implement inference function


In [ ]:
from transformers import AutoTokenizer
import torch
import numpy as np

from transformers import AutoTokenizer
import torch
import numpy as np

def prepare_inference(model_path=None):
    """Initialize tokenizer and load model for inference"""
    tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

    # Load trained model if path provided, otherwise use existing model
    if model_path:
        model = torch.load(model_path)

    id2label = {
        0: "O",
        1: "B-PER",
        2: "I-PER",
        3: "B-ORG",
        4: "I-ORG",
        5: "B-LOC",
        6: "I-LOC",
        7: "B-MISC",
        8: "I-MISC"
    }

    return tokenizer, id2label

def inference(text, model, tokenizer, id2label, device='cuda'):
    """
    Perform NER inference on input text

    Args:
        text (str): Input text to analyze
        model: Trained NER model
        tokenizer: BERT tokenizer
        id2label (dict): Mapping from label ids to label names
        device (str): Device to run inference on ('cuda' or 'cpu')

    Returns:
        list: List of tuples containing (word, entity_label)
    """
    ## fill in your code

    # Ensure model is in evaluation mode


    # Tokenize the text

    # Perform inference


    # Convert predictions to labels


    # Align predictions with words




    # Combine words with their predicted labels


    return labeled_words

def print_entities(labeled_words):
    """Pretty print the labeled entities"""
    current_entity = None
    entity_text = []

    for word, label in labeled_words:
        if label == "O":
            if current_entity:
                print(f"{current_entity}: {' '.join(entity_text)}")
                current_entity = None
                entity_text = []
        elif label.startswith("B-"):
            if current_entity:
                print(f"{current_entity}: {' '.join(entity_text)}")
            current_entity = label[2:]  # Remove "B-" prefix
            entity_text = [word]
        elif label.startswith("I-"):
            if current_entity == label[2:]:  # If it's the same entity type
                entity_text.append(word)
            else:
                if current_entity:
                    print(f"{current_entity}: {' '.join(entity_text)}")
                current_entity = label[2:]
                entity_text = [word]

    if current_entity:  # Print last entity if exists
        print(f"{current_entity}: {' '.join(entity_text)}")

In [ ]:
# First initialize
tokenizer, id2label = prepare_inference()

# Example texts to analyze
texts = [
    "John Smith works at Microsoft in Seattle and visited New York last summer.",
    "The European Union signed a trade deal with Japan in Brussels.",
    "Tesla CEO Elon Musk announced new features coming to their vehicles."
]

# Process each text
for text in texts:
    print("\nText:", text)
    print("Entities found:")
    results = inference(text, model, tokenizer, id2label)
    print_entities(results)


Text: John Smith works at Microsoft in Seattle and visited New York last summer.
Entities found:
PER: John Smith
ORG: Microsoft
LOC: Seattle
LOC: New York

Text: The European Union signed a trade deal with Japan in Brussels.
Entities found:
ORG: European Union
LOC: Japan
LOC: Brussels.

Text: Tesla CEO Elon Musk announced new features coming to their vehicles.
Entities found:
ORG: Tesla
PER: Elon Musk


In [ ]:
# save the model to drive

from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
save_path = "/content/drive/MyDrive/BERT_NER/bert_ner_model.pth"
torch.save(model, save_path)

In [ ]:
model = torch.load(save_path)

<ipython-input-28-f6cbf9c9768c>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(save_path)
